In [1]:
%matplotlib ipympl
import os
import os.path as path
import pandas as pd
import h5py
import numpy as np
from faker import Factory
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *
import plotly
from ipywidgets import fixed, interact_manual, widgets
from ipywidgets import HBox, VBox
from IPython.display import clear_output
from IPython.display import display
from ipywidgets import IntSlider, Output
import plotly.graph_objects as go
import plotly.express as plot_x

# Local module and scripts
from pyccapt.calibration.calibration_tools import tools, data_tools, variables, calibration, data_loadcrop, ion_selection, reconstruction, selectors_data
from pyccapt.calibration.calibration_tools import widgets as wd
from pyccapt.calibration.calibration_tools import dataset_path_qt
from pyccapt.calibration.leap_tools import ccapt_tools

In [2]:
# Disable showing logging in Jupyter notebook
import logging, sys
logging.disable(sys.maxsize)

In [3]:
button = widgets.Button(
    description='load dataset',
)

@button.on_click
def open_file_on_click(b):
    global dataset_path
    dataset_path = dataset_path_qt.gui_fname().decode('ASCII')
button

Button(description='load dataset', style=ButtonStyle())

In [4]:
button_range = widgets.Button(
    description='load range',
)
try:
    del range_path
except:
    pass
@button_range.on_click
def open_file_on_click(b):
    global range_path
    range_path = dataset_path_qt.gui_fname().decode('ASCII')
button_range


Button(description='load range', style=ButtonStyle())

In [7]:
tdc, det_diam, pulse_mode, flightPathLength, t0_d, max_tof = wd.dataset_tdc_selection()
display(tdc, pulse_mode, flightPathLength)

Dropdown(description='TDC model:', options=('surface_concept', 'roentdec', 'leap_epos', 'leap_pos'), value='su…

Dropdown(description='Pulse mode:', options=('voltage', 'laser'), value='voltage')

FloatText(value=110.0, description='Flight path length:')

In [8]:
if 'range_path' in locals():
    range_data = data_tools.read_hdf5_through_pandas(range_path)
    display(range_data)
else:
    range_data = pd.DataFrame({"element": ['unranged'], "isotope": [0], "charge": [0],
        "mc_low": [0],
       "mc_up": [1000],"color": ['#000000']})
    display(range_data)

,element,isotope,charge,mc_low,mc_up,color
0,unranged,0,0,0,1000,#000000


$$\textbf{You can specify which dataset to use in below block}$$

In [9]:
tdc_model = tdc.value
pulse_mode_ini = pulse_mode.value
flight_path_length = flightPathLength.value

dataset_main_path = os.path.dirname(dataset_path)
dataset_name_with_extention = os.path.basename(dataset_path)
dataset_name = os.path.splitext(dataset_name_with_extention)[0]


variables.init()
# variables.path = os.path.join(p, 'tests//data')
variables.result_path = os.path.dirname(dataset_main_path) + '/reconstruction/'
if not os.path.isdir(variables.result_path):
    os.makedirs(variables.result_path, mode=0o777, exist_ok=True)
        
filename = dataset_path

head, tail = os.path.split(filename)
figname = os.path.splitext(tail)[0]

if tdc_model == 'leap_pos' or tdc_model == 'leap_pos':
    if tdc_model == 'leap_epos':
        data = ccapt_tools.epos_to_ccapt(filename)
    else:
        print('The file has to be epos. With pos information this tutorial cannot be run')
        data = ccapt_tools.pos_to_ccapt(filename)
else:
    data = data_tools.read_hdf5_through_pandas(filename)

In [10]:
data

,x (nm),y (nm),z (nm),mc_c (Da),mc (Da),high_voltage (V),pulse (V),start_counter,t (ns),t_c (nm),x_det (cm),y_det (cm),pulse_pi,ion_pp
0,0.0,0.0,0.0,27.463481,26.651289,3738.356445,3738.356445,10164.0,721.125558,0.0,0.299265,-0.264245,0,1
1,0.0,0.0,0.0,27.464963,26.623950,3738.356445,3738.356445,10264.0,721.297008,0.0,0.117796,-0.576245,100,1
2,0.0,0.0,0.0,27.795469,26.947230,3738.356445,3738.356445,10318.0,725.356944,0.0,0.073224,-0.585796,54,1
3,0.0,0.0,0.0,27.163912,26.543927,3738.356445,3738.356445,10384.0,723.923622,0.0,0.397959,1.228898,66,1
4,0.0,0.0,0.0,269.507730,263.962570,3738.356445,3738.356445,10556.0,2164.617972,0.0,-0.630367,0.694041,172,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6252145,0.0,0.0,0.0,26.721781,26.359344,7246.866211,7246.866211,18886.0,551.760390,0.0,-1.980245,-2.753878,152,1
6252146,0.0,0.0,0.0,27.730706,27.402419,7246.866211,7246.866211,18913.0,557.308512,0.0,-1.744653,-2.502367,27,1
6252147,0.0,0.0,0.0,27.913617,28.061340,7246.866211,7246.866211,19050.0,564.276240,0.0,-2.999020,-0.885061,137,1
6252148,0.0,0.0,0.0,27.498398,27.803238,7246.866211,7246.866211,19225.0,547.103808,0.0,-1.149306,0.986939,175,1


In [11]:
# romove data that has x_det and y_det equal to zero at same time
# these points resulted in nan in the reconstruction becuse of dviding 0/0
dld_x = data['x_det (cm)'].to_numpy()
dld_y = data['y_det (cm)'].to_numpy()
mask = np.logical_and((dld_x==0), (dld_y==0))
data.drop(np.where(mask)[0], inplace=True)
data.reset_index(inplace=True, drop=True)

In [12]:
# exctract needed data from Pandas data frame as an numpy array
# dld_highVoltage = data['high_voltage (V)'].to_numpy()
# dld_x = data['x_det (cm)'].to_numpy()
# dld_y = data['y_det (cm)'].to_numpy()
mc = data['mc_c (Da)'].to_numpy()

In [13]:
def mc_hist_plot(mc, bin_size, prominence, distance, percent, selector, plot, figname, lim_mc):
    peaks_ini, peaks_y_ini, peak_widths_p_ini = tools.hist_plot(mc[mc < lim_mc], bin_size, distance=distance, percent=percent, prominence=prominence, selector=selector, plot=plot, label='mc', fig_name=figname+'_ini')
    index_max_ini = np.argmax(peaks_y_ini)
    variables.max_peak = peaks_ini[index_max_ini]
    variables.peak = peaks_ini
    mrp = (peaks_ini[index_max_ini] / (peak_widths_p_ini[index_max_ini][2] - peak_widths_p_ini[index_max_ini][1]))
    print('Mass resolving power for the highest peak (MRP --> m/m_2-m_1):', mrp)
    for i in range(len(peaks_ini)):
        print('Peaks ', i, 'is at location and height: ({:.2f}, {:.2f})'.format(peaks_ini[i], peaks_y_ini[i]), 'peak window sides (half-maximum) are: ({:.2f}, {:.2f})'.format(peak_widths_p_ini[i][1], peak_widths_p_ini[i][2]))

interact_manual(mc_hist_plot, mc=fixed(mc), bin_size=widgets.FloatText(value=0.1)
               ,prominence=widgets.IntText(value=50), distance=widgets.IntText(value=100), lim_mc=widgets.IntText(value=150)
               ,percent=widgets.IntText(value=50), selector=fixed('rect'), plot=fixed(True), figname=fixed(figname));


interactive(children=(FloatText(value=0.1, description='bin_size'), IntText(value=50, description='prominence'…

In [14]:
element_selected = wd.density_field_selection()
display(element_selected)

Dropdown(description='Element', options=((1, 'H', 52.7, 0.0), (2, 'He', 28.7, 0.0), (3, 'Li', 46.3, 14.0), (4,…

In [15]:
avg_dens = element_selected.value[2]
field_evap = element_selected.value[3]

In [19]:
def x_y_z_calculation_and_plot(data, range_data, element_percentage, kf, det_eff, icf, field_evap, avg_dens, rotary_fig_save, selected_are, figname):
    dld_highVoltage = data['high_voltage (V)'].to_numpy()
    dld_x = data['x_det (cm)'].to_numpy()
    dld_y = data['y_det (cm)'].to_numpy()
    mc = data['mc_c (Da)'].to_numpy()
    flight_path_length = flightPathLength.value
    px, py, pz = reconstruction.atom_probe_recons_from_detector_Gault_et_al(dld_x, dld_y, dld_highVoltage,
                                                                            flight_path_length, kf, det_eff, icf,
                                                                            field_evap, avg_dens)
    #     px, py, pz = reconstruction.atom_probe_recons_Bas_et_al(dld_x, dld_y, dld_highVoltage, flight_path_length, kf, det_eff, icfe, field_evap, avg_dens)
    data['x (nm)'] = px
    data['y (nm)'] = py
    data['z (nm)'] = pz
    reconstruction.reconstruction_plot(data, range_data, element_percentage, rotary_fig_save, selected_are, figname)

In [21]:
if range_data is not None:
    element_percentage = [0.01] * len(range_data['element'].tolist())
    element_percentage = str(element_percentage)
else:
    element_percentage = str([0.01])

interact_manual(x_y_z_calculation_and_plot, kf=widgets.FloatText(value=4), det_eff=widgets.FloatText(value=0.7), icf=widgets.FloatText(value=1.4),
                field_evap=widgets.FloatText(value=field_evap),
               avg_dens=widgets.FloatText(value=avg_dens), element_percentage=widgets.Textarea(value=element_percentage),
               rotary_fig_save=widgets.Dropdown(options=[('True', True), ('False', False)], value=False), data=fixed(data), range_data=fixed(range_data), selected_are=fixed(False), figname=figname);

C:\Users\APTUser\AppData\Local\Temp\ipykernel_53000\939047964.py:7: DeprecationWarning:

on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').



interactive(children=(Textarea(value='[0.01]', description='element_percentage'), FloatText(value=4.0, descrip…

In [22]:
data

,x (nm),y (nm),z (nm),mc_c (Da),mc (Da),high_voltage (V),pulse (V),start_counter,t (ns),t_c (nm),x_det (cm),y_det (cm),pulse_pi,ion_pp
0,1.129020,-0.996901,0.038280,27.463481,26.651289,3738.356445,3738.356445,10164.0,721.125558,0.0,0.299265,-0.264245,0,1
1,0.443923,-2.171625,0.082964,27.464963,26.623950,3738.356445,3738.356445,10264.0,721.297008,0.0,0.117796,-0.576245,100,1
2,0.275948,-2.207586,0.083608,27.795469,26.947230,3738.356445,3738.356445,10318.0,725.356944,0.0,0.073224,-0.585796,54,1
3,1.488415,4.596224,0.396092,27.163912,26.543927,3738.356445,3738.356445,10384.0,723.923622,0.0,0.397959,1.228898,66,1
4,-2.368316,2.607539,0.209980,269.507730,263.962570,3738.356445,3738.356445,10556.0,2164.617972,0.0,-0.630367,0.694041,172,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6252142,-13.607236,-18.923247,71.267921,26.721781,26.359344,7246.866211,7246.866211,18886.0,551.760390,0.0,-1.980245,-2.753878,152,1
6252143,-12.128276,-17.395666,70.385816,27.730706,27.402419,7246.866211,7246.866211,18913.0,557.308512,0.0,-1.744653,-2.502367,27,1
6252144,-20.796038,-6.137260,70.577299,27.913617,28.061340,7246.866211,7246.866211,19050.0,564.276240,0.0,-2.999020,-0.885061,137,1
6252145,-8.303195,7.130168,67.384534,27.498398,27.803238,7246.866211,7246.866211,19225.0,547.103808,0.0,-1.149306,0.986939,175,1


In [23]:

plot_3d_button = widgets.Button(
    description='plot 3D',
)
plot_heatmap_button = widgets.Button(
    description='plot heatmap',
)
plot_mc_button = widgets.Button(
    description='plot mc',
)
plot_projection_button = widgets.Button(
    description='plot projection',
)

clear_button = widgets.Button(
    description='Clear plots',
)

selected_area_p3=widgets.Dropdown(options=[('False', False), ('True', True)], description='Selected area')
rotary_fig_save_p3=widgets.Dropdown(options=[('True', True), ('False', False)], description='Rotary save')
element_percentage_p3=widgets.Textarea(value=element_percentage, description='Element percentage')
@plot_3d_button.on_click
def plot_3d(b, data=data, range_data=range_data, figname=figname):
    with out:
        if not variables.selected_calculated:
            variables.selected_z1 = variables.selected_y1
            variables.selected_z2 = variables.selected_y2
            variables.selected_y1 = variables.selected_x1
            variables.selected_y2 = variables.selected_x2
            print('Min x (nm):', variables.selected_x1, 'Max x (nm):', variables.selected_x2)
            print('Min y (nm):', variables.selected_y1, 'Max y (nm):', variables.selected_y2)
            print('Min z (nm):', variables.selected_z1, 'Max z (nm):', variables.selected_z2)
            variables.selected_calculated = True
        reconstruction.reconstruction_plot(data, range_data, element_percentage_p3.value, rotary_fig_save_p3.value, selected_area_p3.value, figname)

selected_area_ph=widgets.Dropdown(options=[('False', False), ('True', True)], description='Selected area')
element_percentage_ph=widgets.Textarea(value=element_percentage, description='Element percentage')
@plot_heatmap_button.on_click
def plot_heatmap(b, data=data, range_data=range_data):
    with out:
        if not variables.selected_calculated:
            variables.selected_z1 = variables.selected_y1
            variables.selected_z2 = variables.selected_y2
            variables.selected_y1 = variables.selected_x1
            variables.selected_y2 = variables.selected_x2
            print('Min x (nm):', variables.selected_x1, 'Max x (nm):', variables.selected_x2)
            print('Min y (nm):', variables.selected_y1, 'Max y (nm):', variables.selected_y2)
            print('Min z (nm):', variables.selected_z1, 'Max z (nm):', variables.selected_z2)
            variables.selected_calculated = True
        data_f = data.copy(deep=True)
        if selected_area_ph.value:
            data_f = data_f[(data_f['x (nm)'] >  variables.selected_x1) & (data_f['x (nm)'] <  variables.selected_x2) & 
                            (data_f['y (nm)'] >  variables.selected_y1) & (data_f['y (nm)'] <  variables.selected_y2) & 
                            (data_f['z (nm)'] >  variables.selected_z1) & (data_f['z (nm)'] <  variables.selected_z2)]

            data_f.reset_index(inplace=True, drop=True)
        reconstruction.heatmap(data_f, range_data, selected_area_ph.value, element_percentage_ph.value, save=True)

selected_area_pm=widgets.Dropdown(options=[('False', False), ('True', True)], description='Selected area')
range_file_exist_pm=widgets.Dropdown(options=[('True', True), ('False', False)], description='Range exist')
bin_size_pm = widgets.FloatText(value=0.1, description='Bins size')
prominence_pm = widgets.IntText(value=50, description='Prominance')
distance_pm = widgets.IntText(value=100, description='Distance')
lim_mc_pm = widgets.IntText(value=150, description='Limit mc')
percent_pm = widgets.IntText(value=50, description='Percent MRP')
@plot_mc_button.on_click
def plot_mc(b, data=data, range_data=range_data):
    with out:
        if not variables.selected_calculated:
            variables.selected_z1 = variables.selected_y1
            variables.selected_z2 = variables.selected_y2
            variables.selected_y1 = variables.selected_x1
            variables.selected_y2 = variables.selected_x2
            print('Min x (nm):', variables.selected_x1, 'Max x (nm):', variables.selected_x2)
            print('Min y (nm):', variables.selected_y1, 'Max y (nm):', variables.selected_y2)
            print('Min z (nm):', variables.selected_z1, 'Max z (nm):', variables.selected_z2)
            variables.selected_calculated = True
        data_f = data.copy(deep=True)
        if selected_area_pm.value:
            data_f = data_f[(data_f['x (nm)'] >  variables.selected_x1) & (data_f['x (nm)'] <  variables.selected_x2) & 
                            (data_f['y (nm)'] >  variables.selected_y1) & (data_f['y (nm)'] <  variables.selected_y2) & 
                            (data_f['z (nm)'] >  variables.selected_z1) & (data_f['z (nm)'] <  variables.selected_z2)]

            data_f.reset_index(inplace=True, drop=True)
        mc = data_f['mc_c (Da)'].to_numpy()

        peaks_ini, peaks_y_ini, peak_widths_p_ini = tools.hist_plot(mc[mc < lim_mc_pm.value], range_data=range_data, ranging=range_file_exist_pm.value, bin=bin_size_pm.value, distance=distance_pm.value, percent=percent_pm.value,
                                                                    prominence=prominence_pm.value, selector=None, plot=True, label='mc', fig_name=figname+'_ini', fast_hist=False)
        index_max_ini = np.argmax(peaks_y_ini)
        mrp = (peaks_ini[index_max_ini] / (peak_widths_p_ini[index_max_ini][2] - peak_widths_p_ini[index_max_ini][1]))
        print('Mass resolving power for the highest peak (MRP --> m/m_2-m_1):', mrp)
        for i in range(len(peaks_ini)):
            print('Peaks ', i, 'is at location and height: ({:.2f}, {:.2f})'.format(peaks_ini[i], peaks_y_ini[i]), 'peak window sides (half-maximum) are: ({:.2f}, {:.2f})'.format(peak_widths_p_ini[i][1], peak_widths_p_ini[i][2]))

element_percentage_pp=widgets.Textarea(value=element_percentage, description='Element percentage')
selected_area_pp=widgets.Dropdown(options=[('False', False), ('True', True)], description='Selected area')
x_or_y_pp=widgets.Dropdown(options=['x', 'y'], value='x', description='X or Y')

@plot_projection_button.on_click
def plot_projection(b, data=data, range_data=range_data, figname=figname):
    with out:
        if not variables.selected_calculated:
            variables.selected_z1 = variables.selected_y1
            variables.selected_z2 = variables.selected_y2
            variables.selected_y1 = variables.selected_x1
            variables.selected_y2 = variables.selected_x2
            print('Min x (nm):', variables.selected_x1, 'Max x (nm):', variables.selected_x2)
            print('Min y (nm):', variables.selected_y1, 'Max y (nm):', variables.selected_y2)
            print('Min z (nm):', variables.selected_z1, 'Max z (nm):', variables.selected_z2)
            variables.selected_calculated = True
        data_f = data.copy(deep=True)
        if selected_area_pp.value:
            data_f = data_f[(data_f['x (nm)'] >  variables.selected_x1) & (data_f['x (nm)'] <  variables.selected_x2) & 
                            (data_f['y (nm)'] >  variables.selected_y1) & (data_f['y (nm)'] <  variables.selected_y2) & 
                            (data_f['z (nm)'] >  variables.selected_z1) & (data_f['z (nm)'] <  variables.selected_z2)]
        
            data_f.reset_index(inplace=True, drop=True)
        reconstruction.projection(data_f, range_data, element_percentage_pp.value, selected_area_pp.value, x_or_y_pp.value, figname)

@clear_button.on_click
def clear(b,):
    with out:
        clear_output(True)
        print('')


tab1 = VBox(children=[selected_area_pp, x_or_y_pp, element_percentage_pp, plot_projection_button, clear_button])
tab2 = VBox(children=[selected_area_p3, rotary_fig_save_p3, element_percentage_p3, plot_3d_button, clear_button])
tab3 = VBox(children=[selected_area_pm, bin_size_pm, prominence_pm, distance_pm, lim_mc_pm, percent_pm, plot_mc_button, clear_button])
tab4 = VBox(children=[selected_area_ph, element_percentage_ph, plot_heatmap_button, clear_button])
# tab2 = VBox(children=[VBox(children=[HBox(children=[plot_3d_button, selected_area_p3, range_file_exist_p3, rotary_fig_save_p3, element_percentage_p3]),
#                                      HBox(children=[plot_heatmap_button, selected_area_ph, element_percentage_ph]),
#                                      HBox(children=[plot_mc_button, selected_area_pm, range_file_exist_pm, bin_size_pm, prominence_pm, distance_pm, lim_mc_pm, percent_pm]), 
#                                      HBox(children=[plot_projection_button, selected_area_pp, x_or_y_pp, element_percentage_pp])]), clear_button])


tab = widgets.Tab(children=[tab1, tab2, tab3, tab4])
tab.set_title(0, 'projection')
tab.set_title(1, '3d plot')
tab.set_title(2, 'mc plot')
tab.set_title(3, 'heatmap plot')



display(VBox(children=[tab]))
out = Output()
display(out)

Output()

In [24]:
# save the new data
name_save_file = variables.result_path + '//' + dataset_name + '.h5'
data_tools.store_df_to_hdf(name_save_file, data, 'df')

In [25]:
# save data as epos file
epos = ccapt_tools.ccapt_to_epos(data, path=variables.result_path, name=dataset_name + '.epos')
pos = ccapt_tools.ccapt_to_pos(data, path=variables.result_path, name=dataset_name + '.pos')

In [85]:
# save data in csv format
data_tools.store_df_to_csv(data, variables.result_path + dataset_name + '.csv')